In [6]:
import numpy as np
!pip install qutip
from qutip import *
import matplotlib.pyplot as plt

# Define Pauli matrices for two-qubit system
sx = sigmax()  # Pauli-X for one qubit
sy = sigmay()  # Pauli-Y for one qubit
sz = sigmaz()  # Pauli-Z for one qubit
sm = destroy(2)  # Lowering operator (|1><0|)

# Tensor product for two-qubit operators
# Bell state: Psi- = (|01> - |10>) / sqrt(2)
psi_bell = (basis(2, 0) * basis(2, 1).dag() - basis(2, 1) * basis(2, 0).dag()).unit()

# Initial state
rho0 = psi_bell * psi_bell.dag()  # Density matrix of the initial state

# Hamiltonian (can be set to zero for pure decoherence)
H = 0 * tensor(sx, sx)  # No unitary evolution (just decoherence here)

# Define Lindblad operators (noise)
# Decoherence (dephasing)
L_dephasing_1 = np.sqrt(0.2) * tensor(sz, qeye(2))
L_dephasing_2 = np.sqrt(0.2) * tensor(qeye(2), sz)

# Amplitude damping
L_amplitude_1 = np.sqrt(0.1) * tensor(sm, qeye(2))
L_amplitude_2 = np.sqrt(0.1) * tensor(qeye(2), sm)

# Time points for evolution
tlist = np.linspace(0, 10, 100)

# Solve the Lindblad master equation
result = mesolve(H, rho0, tlist, [L_dephasing_1, L_dephasing_2, L_amplitude_1, L_amplitude_2])

# Get the density matrices at each time step
rho_t = result.states

# Plot the evolution of the expectation value of some observable (optional)
expectations = expect(tensor(sz, qeye(2)), rho_t)
plt.plot(tlist, expectations)
plt.xlabel("Time")
plt.ylabel(r"$\langle \sigma_z \rangle$")
plt.show()

# Display the first few density matrices as an example
for i in range(3):
    print(f"Density matrix at t = {tlist[i]}:\n{rho_t[i]}\n")

  Using cached qutip-5.0.1.tar.gz (6.4 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  pip subprocess to install build dependencies did not run successfully.
  exit code: 1
  
  [14 lines of output]
  Ignoring cython: markers 'python_version >= "3.10"' don't match your environment
    Using cached setuptools-68.0.0-py3-none-any.whl.metadata (6.4 kB)
    Using cached packaging-24.0-py3-none-any.whl.metadata (3.2 kB)
    Using cached wheel-0.42.0-py3-none-any.whl.metadata (2.2 kB)
    Using cached Cython-3.0.2-cp37-cp37m-win_amd64.whl.metadata (3.2 kB)
    Using cached oldest_supported_numpy-2023.12.21-py3-none-any.whl.metadata (9.8 kB)
  ERROR: Ignored the following versions that require a different python version: 0.43.0 Requires-Python >=3.8; 0.44.0 Requires-Python >=3.8; 1.10.0 Requires-Python <3.12,>=3.8; 1.10.0rc1 Requires-Python <3.12,>=3.8; 1.10.0rc2 Requires-Python <3.12,>=3.8; 1.10.1 Requires-Python <3.12,>=3.8; 1.11.0 Requires-Python <3.13,>=3.9; 1.11.0rc1 Requires-Python <3.13,>=3.9; 1.11.0rc2 Requires-Python <3.13,>=3

ModuleNotFoundError: No module named 'qutip'

In [7]:
# Calculate concurrence for each density matrix
concurrences = [concurrence(rho) for rho in rho_t]

# Classify as entangled (concurrence > 0) or separable (concurrence = 0)
entanglement_labels = [1 if conc > 0 else 0 for conc in concurrences]

# Print the first few concurrences and labels
for i in range(3):
    print(f"Time: {tlist[i]:.2f}, Concurrence: {concurrences[i]:.3f}, Label (Entangled: 1, Separable: 0): {entanglement_labels[i]}")

NameError: name 'rho_t' is not defined

In [ ]:
# Calculate expectation values of Pauli matrices for each qubit
def calculate_expectation_values(rho):
    # Pauli matrices for the first and second qubits
    sx1, sy1, sz1 = tensor(sigmax(), qeye(2)), tensor(sigmay(), qeye(2)), tensor(sigmaz(), qeye(2))
    sx2, sy2, sz2 = tensor(qeye(2), sigmax()), tensor(qeye(2), sigmay()), tensor(qeye(2), sigmaz())
    
    # Calculate expectation values for each Pauli matrix
    exp_sx1 = expect(sx1, rho)
    exp_sy1 = expect(sy1, rho)
    exp_sz1 = expect(sz1, rho)
    exp_sx2 = expect(sx2, rho)
    exp_sy2 = expect(sy2, rho)
    exp_sz2 = expect(sz2, rho)
    
    return [exp_sx1, exp_sy1, exp_sz1, exp_sx2, exp_sy2, exp_sz2]

# Update the feature extraction function to include expectation values
def extract_features_with_expectations(rho):
    # Calculate basic features: purity, entropy, eigenvalues
    purity = (rho * rho).tr().real  # Purity of the state
    entropy = entropy_vn(rho).real  # Von Neumann entropy
    eigvals = rho.eigenenergies()   # Eigenvalues of the density matrix
    
    # Calculate expectation values of Pauli matrices
    expectation_values = calculate_expectation_values(rho)
    
    # Combine all features: purity, entropy, eigenvalues, and expectation values
    return [purity, entropy] + list(eigvals) + expectation_values

# Extract features (with expectation values) for each density matrix in the time evolution
features_with_expectations = [extract_features_with_expectations(rho) for rho in rho_t]

# Convert to NumPy array for easier handling in machine learning
features_with_expectations = np.array(features_with_expectations)

# Print the first few feature sets with expectation values
print(f"Features for first density matrix (with expectation values):\n{features_with_expectations[0]}")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_with_expectations, entanglement_labels, test_size=0.3, random_state=42)

# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Detailed classification report
print(classification_report(y_test, y_pred, target_names=["Separable", "Entangled"]))
